# Artificial Intelligence - CA2
## Hossein Soltanloo - 810195407

First the needed libraries are included.

In [1]:
import re
import string
import random
import copy
from nltk.tokenize import word_tokenize
ALPHABET = "abcdefghijklmnopqrstuvwxyz"

#### Cleaning the Data
In order to clean the raw data used as training text, we follow a few steps as follows:
1. Read the raw data file
2. Lowercase all the letters
3. Replace numbers with blank spaces
4. Replace punctuations with blank spaces
5. Tokenize the text into words
6. Create a set both to delete repeated words and have a faster way to search among them

These steps are done in order to extract only the words and not punctuations and numbers; because only the words have value for us. We use them in order to check if a decrypted word has a meaning and is present in our dictionary.

In [2]:
def read_file(file_path):
    with open(file_path, 'r') as myfile:
        return myfile.read()
    
def clean_text(text):
    new_text = text.lower()
    new_text = re.sub(r"\d+", " ", new_text)
    for ch in string.punctuation:                                                                                                     
        new_text = new_text.replace(ch, " ")
    new_text = ' '.join(new_text.split())
    word_tokens = word_tokenize(new_text) 
    data = new_text.split()
    return set(word_tokens)

#### Chromosome
I define a chromosome as a mapping from the normal alphabet to their substitutions. As a helper tool, I have defined the following method to create a dictionary from a string. The string is used as a shortened representation of the key where the letters are substitutions of the original alphabet with regards to their order.

In [3]:
def create_dict_key(key_string):
    dict_key = dict()
    for i in range(len(ALPHABET)):
        dict_key[ALPHABET[i]] = key_string[i]
    return dict_key

To generate children from two parents, we first need to initialize an empty mapping and fill it with the crossover and mutation operations. The following method helps to initialize the children:

In [4]:
def create_empty_dict():
    return {chr(i):'' for i in range(97, 123)}

The following function takes a potential mapping and decrypts the given text:

In [5]:
def decrypt(cipher_text, key_dict):
    decrypted_text = ""
    for letter in cipher_text:
        uppercase = letter.isupper()
        if uppercase:
            decrypted_text += key_dict[letter.lower()].upper()
        else:
            if letter in ALPHABET:
                decrypted_text += key_dict[letter]
            else:
                decrypted_text += letter

    return decrypted_text

#### Fitness
The fitness of a chromosome is a value with one characteristic, it increases as the chromosome better maps the cipher text to have more meaningful content. The value itself is irrelevant. We can, however, use the value for comparative purposes, assuming the fitness is calculated with the same cipher text for both chromosomes. After performing this comparison, the chromosome with the higher fitness is the "better" chromosome.

I have used the number of meaningful words in a deciphered text as the measure of fitness. By meaningful words, I mean words that are present in the initial dictionary extracted from the training text.

In [6]:
def calculate_fitness(decrypted_text, dictionary):
    fitness = 0
    for word in decrypted_text:
        if word in dictionary or word in ALPHABET:
            fitness += 1
    return fitness

#### Crossover and Mutation
Crossover generates two child chromosomes from two parent chromosomes, each with roughly half the mappings of either parent. This is a good way to ensure that the best parts of different chromosomes are mixed and that multiple chromosomes with different correct mappings can transform into one chromosome with many correct mappings after only a few generations. This is performed by the function `crossover`, which takes two chromosomes to act as parents. It creates two new chromosomes, `child1` and `child2`. These children are initially empty. A random slicing point is generated and then the first part of the first child is filled with the first slice of the first parent and the second slice is filled wi the second slice of the second parent. There may be some mappings of this second slice that might have already been filled by the first parent, so we ignore them and solve this problem by filling the blank spaces with the remaining alphabet. The same is done for the second child with the first and second parents swapped.

Mutation generates a child chromosome from one parent chromosome, with multiple mappings swapped. For example, if a chromosome contained the mappings (a, b) and (c, d), and these mappings were the ones selected for mutation, then the resulting mappings in the child would be (a, d) and (c, b). This type of child allows another method for new genes to be introduced into successful genes, improving the genetic diversity of the population.\
The mutation is done with a %70 chance to control the convergance of chromosomes into better ones and not mutate every child. I mutate the children in variant numbers to add more diversity into the chromosomes. In normal conditions, from 1 to 5 mutations are applied to the children. There is also another condition where we need to refresh the population when it has been stuck and not improving for many generations. In this condition we need to shock the chromosomes to have more chance to get out of the local optima and for this to happen, I apply more mutations (10-20) to each child. If more than a certain number of generations have had the same maximum fitness, then a shock will be applied.\
I used the genetic shock when I had problems with my project and it was getting stuck in local optima. I did other optimizations and removed this condition and it has seemed to be working well without it. So that's just a potential solution to the problem of getting stuck in a local optima.

In [7]:
def crossover(parent_key1, parent_key2, shock = False):
    parent_key1 = create_dict_key(parent_key1)
    parent_key2 = create_dict_key(parent_key2)

    slice_index = random.randrange(0, 26)
    child1 = merge_parents(parent_key1,parent_key2,slice_index)
    child2 = merge_parents(parent_key2,parent_key1,slice_index)

    is_mutated = random.randrange(0, 10)
    if is_mutated < 7 or shock:
        child1 = mutate_child(''.join(child1), shock)
        child2 = mutate_child(''.join(child2), shock)

    child1 = ''.join(child1)
    child2 = ''.join(child2)
    return [child1, child2]

def merge_parents(parent_key1, parent_key2, slice_index):
    child = create_empty_dict()
    
    for i in range(0, slice_index):
        current_letter = chr(i + 97)
        child[current_letter] = parent_key1[current_letter]

    for j in range(slice_index, len(parent_key2)):
        current_letter = chr(j + 97)
        if parent_key2[current_letter] not in child.values() and child[current_letter] == '':
            child[current_letter] = parent_key2[current_letter]

    shuffled_alhpabet = copy.deepcopy(list(ALPHABET))
    random.shuffle(shuffled_alhpabet)
    for a in shuffled_alhpabet:
        for k in child:
            if a not in child.values() and child[k] == '':
                child[k] = a
                
    child = list(child.values())
    return child

def mutate_child(original, shock):
    mutated_child = create_dict_key(original)
    if shock:
        mutations = random.randint(10,20)
    else:
        mutations = random.randint(1,5)
    for i in range(0, mutations):
        idx1 = random.randint(0,25)
        idx2 = random.randint(idx1,25)
        temp = mutated_child[chr(idx1 + 97)]
        mutated_child[chr(idx1 + 97)] = mutated_child[chr(idx2 + 97)]
        mutated_child[chr(idx2 + 97)] = temp    
    return list(mutated_child.values())

#### Running the Algorithm
I have set the initial population size to be 1000 in order to have a better and more diverse start. For every population, I calculate fitness for all chromosomes in that population and sort the chromosomes by their fitnesses. Afterwards, in the selection phase, I choose the 50 most fit parents to generate children. They're called "Elites". Child generation is done in this way: Each elite parent generates two children by a %60 chance with the next 5 parents which are ordered by their fitness. In addition to the new children, the elites are carried to the next generation to maintain their strong fitness in the next generation and use them if the new children are not strong enough. In special conditions when we need a genetic shock to inject fresh genes into the population, 1000 children are generated from 500 random pairs of parents which are selected from the whole population and not just the elites. This way we can add more diversity and be able to exit from the local optima.

Answers to the questions in the project description:
- If we increase the population size, this makes it harder for a locally optimized chromosome to overpower the entire genetic pool, and makes the locally optimized chromosome more likely to encounter a chromsome that will break it out of the local optima as it spreads. It will optimize the combination of speed and accuarcy but we cannot say for sure if either will be optimized individually.

- If we use crossover without mutation, then the algorithm won't be able to effectively create new children with better fitnesses. The algorithm would be generating the best children it can but after some generations, it would always be recombining the same parents and generating children that are just like their parents and are not any better. Mutation is used to introduce diversity to the population. Hence if not used, we would be soon facing a local optima that can't be escaped.

- I think that a combination of both is important and there is no point in using them individually. But we can say that a crossover is more important in generating fitter children and a mutation is important in keeping the population diverse. Both of these methods combine to make it work.

- As mentioned earlier, we can do several things to prevent a convergence. A convergence may happen due to a less diverse population, a low populations size, low crossover chances, and poor selection methods. In order to overcome a local optimum, we can use genetic shocks, more mutations, and more population size. All of them are already described above and I have used a combination of the solutions to overcome these issues.

In [8]:
MAX_GENERATIONS = 5000
ELITES_SIZE = 50
INITIAL_POPULATION = 1000
GENERATION_SIZE = 500
MAX_REPETITIONS = 5

def sort_dict(dictionary):
    sorted_dict_list = sorted(copy.deepcopy(dictionary), key=dictionary.get, reverse=True)
    return sorted_dict_list

def genetic_algorithm(cipher_text):
    cleaned_data = clean_text(read_file('Attachment/global_text.txt'))
    gen_num = 0
    hold_text = {}
    parents = [''.join(random.sample(ALPHABET, len(ALPHABET))) for i in range(INITIAL_POPULATION)]
    last_top_fitness = 0
    repeated_fitness = 0

    while gen_num < MAX_GENERATIONS:
        decryptions = {}
        for parent in parents:
            parent_key = create_dict_key(copy.deepcopy(parent))
            decrypted_text = decrypt(cipher_text, parent_key)
            cleaned_decrypted_text = clean_text(decrypted_text)
            fitness = calculate_fitness(cleaned_decrypted_text, cleaned_data)
            decryptions[parent] = fitness
            hold_text[parent] = decrypted_text
            if fitness == len(cleaned_decrypted_text):
                return parent, hold_text[parent]

        parents_sorted = sort_dict(copy.deepcopy(decryptions))
        if last_top_fitness == decryptions[parents_sorted[0]]:
            repeated_fitness += 1
        else:
            repeated_fitness = 0
            last_top_fitness = decryptions[parents_sorted[0]]
        
        new_generation = []

        for i in range(0, ELITES_SIZE):
            for j in range(0, 4):
                parent1 = parents_sorted[i]
                parent2 = parents_sorted[i+1]
                chance = random.randint(0, 9)
                if chance > 3:
                    new_generation += crossover(parent1, parent2)

        parents = parents_sorted[0:ELITES_SIZE] + new_generation[:GENERATION_SIZE - ELITES_SIZE]
        
        if repeated_fitness >= MAX_REPETITIONS:
            if repeated_fitness % MAX_REPETITIONS == 0:
                for i in range(0, GENERATION_SIZE):
                    idx1 = random.randint(0, ELITES_SIZE - 1)
                    idx2 = random.randint(0, ELITES_SIZE - 1)
                    parent1 = parents_sorted[idx1]
                    parent2 = parents_sorted[idx2]
                    parents += crossover(parent1, parent2)

        gen_num += 1

After running it for 5 times, the algorithm averaged for 80 seconds.

In [9]:
import time
times = []
for i in range(0, 5):
    start = time.process_time()
    key = genetic_algorithm(read_file('Attachment/encoded_text.txt'))
    times.append(time.process_time() - start)
print("Average running time for 5 executions:  ",sum(times)/len(times))

Average running time for 5 executions:   80.33116720000001
